<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-0.1">Imports</a></span></li><li><span><a href="#Load-Subset-of-Data-into-Pandas-Dataframe" data-toc-modified-id="Load-Subset-of-Data-into-Pandas-Dataframe-0.2">Load Subset of Data into Pandas Dataframe</a></span></li><li><span><a href="#Limit-Data-to-Subset-of-Users-(applies-to-orders,-and-order_products)" data-toc-modified-id="Limit-Data-to-Subset-of-Users-(applies-to-orders,-and-order_products)-0.3">Limit Data to Subset of Users (applies to <code>orders</code>, and <code>order_products</code>)</a></span></li></ul></li><li><span><a href="#Single-Dataframe-EDA" data-toc-modified-id="Single-Dataframe-EDA-1">Single Dataframe EDA</a></span><ul class="toc-item"><li><span><a href="#EDA-–-Products" data-toc-modified-id="EDA-–-Products-1.1">EDA – Products</a></span></li><li><span><a href="#EDA-–-Aisles" data-toc-modified-id="EDA-–-Aisles-1.2">EDA – Aisles</a></span></li><li><span><a href="#EDA-–-Depts" data-toc-modified-id="EDA-–-Depts-1.3">EDA – Depts</a></span></li><li><span><a href="#EDA-–-Orders" data-toc-modified-id="EDA-–-Orders-1.4">EDA – Orders</a></span></li><li><span><a href="#EDA---Order-Products" data-toc-modified-id="EDA---Order-Products-1.5">EDA - Order Products</a></span><ul class="toc-item"><li><span><a href="#Prior" data-toc-modified-id="Prior-1.5.1">Prior</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-1.5.2">Train</a></span></li></ul></li><li><span><a href="#Merge-Prior-and-Train-Orders" data-toc-modified-id="Merge-Prior-and-Train-Orders-1.6">Merge Prior and Train Orders</a></span></li></ul></li><li><span><a href="#User-Database-and-Features" data-toc-modified-id="User-Database-and-Features-2">User Database and Features</a></span><ul class="toc-item"><li><span><a href="#Initialize-User-Dataframe" data-toc-modified-id="Initialize-User-Dataframe-2.1">Initialize User Dataframe</a></span></li><li><span><a href="#Generate-High-level-User-Features" data-toc-modified-id="Generate-High-level-User-Features-2.2">Generate High-level User Features</a></span><ul class="toc-item"><li><span><a href="#Unique-Order-Count-(User)" data-toc-modified-id="Unique-Order-Count-(User)-2.2.1">Unique Order Count (User)</a></span></li><li><span><a href="#Order-Sizes-(User)" data-toc-modified-id="Order-Sizes-(User)-2.2.2">Order Sizes (User)</a></span></li><li><span><a href="#Average-Products-/-Order-(User)" data-toc-modified-id="Average-Products-/-Order-(User)-2.2.3">Average Products / Order (User)</a></span></li><li><span><a href="#Departmental-Unique-Check-ins-Dictionary-(User)" data-toc-modified-id="Departmental-Unique-Check-ins-Dictionary-(User)-2.2.4">Departmental Unique Check-ins Dictionary (User)</a></span></li><li><span><a href="#Total-Orders-to-Date-(User)" data-toc-modified-id="Total-Orders-to-Date-(User)-2.2.5">Total Orders to Date (User)</a></span></li><li><span><a href="#Unique-Orders-(User)" data-toc-modified-id="Unique-Orders-(User)-2.2.6">Unique Orders (User)</a></span></li></ul></li></ul></li></ul></div>

## Imports

In [39]:
import math
import random
import pandas as pd
import numpy as np
import itertools as IT

import os
import sys

## Load Subset of Data into Pandas Dataframe

In [7]:
aisles = pd.read_csv('../data/instacart-market-basket-analysis/aisles.csv')
print(aisles.shape)
depts = pd.read_csv('../data/instacart-market-basket-analysis/departments.csv')
print(depts.shape)
prods = pd.read_csv('../data/instacart-market-basket-analysis/products.csv')
print(prods.shape)

(134, 2)
(21, 2)
(49688, 4)


## Limit Data to Subset of Users (applies to `orders`, and `order_products`) 

In [56]:
# define subset of data
# Take a sample of the 10,000 users represented in the data

# randomized version
# np.random.seed(4444)
# user_sample_list = np.random.choice(a=range(1,10000,1), size=300, replace=False)

user_sample_list = range(1,501, 1)

print('generated sample of size ', len(user_sample_list))

generated sample of size  500


In [15]:
# Create a generalized mask that filters rows
# which are not applicable to the sampled users.
#
# masked_df = pd.DataFrame(
#     columns=['user_id']
# )
#
# user_mask = ([
#     (masked_df.user_id.isin(user_sample_list))
# ])

orders = pd.DataFrame(
    columns=['order_id','user_id','eval_set','order_number',
             'order_dow','order_hour_of_day','days_since_prior_order']
)

# import the orders csv in chunks
# filter out rows that aren't from the user sample
# concatenate to the prior/train dataset


def valid_user(chunks):
    for chunk in chunks:
        mask = chunk.user_id.isin(user_sample_list)
        yield chunk.loc[mask]

In [25]:
# read in orders
filename = '../data/instacart-market-basket-analysis/orders.csv'
chunksize = 10 ** 4
chunks = pd.read_csv(filename, chunksize=chunksize)
orders = pd.concat(valid_user(chunks))

In [51]:
print(sys.getsizeof(orders)/1000000,'MB vs ',
      os.stat('../data/instacart-market-basket-analysis/orders.csv').st_size//1000000,'MB')

1.288284 MB vs  108 MB


In [57]:
# compile a list of unique orders that originate from our user sample

orders_sample_list = orders.order_id.unique()
print(orders_sample_list.shape)

def valid_order(chunks):
    for chunk in chunks:
        mask = chunk.order_id.isin(orders_sample_list)
        yield chunk.loc[mask]

(8142,)


In [50]:
order_prods__prior = pd.DataFrame(
    columns=['order_id','product_id','add_to_cart_order','reordered']
)

# import the prior/train_products csv in chunks
# filter out rows that aren't from the user sample
# concatenate to the prior/train dataset



# read in order_prods_prior
filename = '../data/instacart-market-basket-analysis/order_products__prior.csv'
chunksize = 10 ** 4
chunks = pd.read_csv(filename, chunksize=chunksize)
order_prods__prior = pd.concat(valid_order(chunks))

In [53]:
print(sys.getsizeof(order_prods__prior)/1000000,'MB vs ',
      os.stat('../data/instacart-market-basket-analysis/order_products__prior.csv').st_size//1000000,'MB')

3.097032 MB vs  577 MB


In [54]:
order_prods__train = pd.DataFrame(
    columns=['order_id','product_id','add_to_cart_order']
)

# import the prior/train_products csv in chunks
# filter out rows that aren't from the user sample
# concatenate to the prior/train dataset

# read in order_prods_prior
filename = '../data/instacart-market-basket-analysis/order_products__train.csv'
chunksize = 10 ** 4
chunks = pd.read_csv(filename, chunksize=chunksize)
order_prods__train = pd.concat(valid_order(chunks))

In [55]:
print(sys.getsizeof(order_prods__train)/1000000,'MB vs ',
      os.stat('../data/instacart-market-basket-analysis/order_products__train.csv').st_size//1000000,'MB')

0.135472 MB vs  24 MB


# Single Dataframe EDA

## EDA – Products

In [61]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


## EDA – Aisles

In [60]:
aisles.head(3)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


## EDA – Depts

In [63]:
depts.head(3)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery


## EDA – Orders

In [64]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


## EDA - Order Products

### Prior

In [108]:
print(order_prods__prior.shape)
order_prods__prior.head()

(77425, 4)


,order_id,product_id,add_to_cart_order,reordered
333,40,10070,1,1
334,40,42450,2,1
335,40,33198,3,1
336,40,34866,4,1
14111,1483,32818,1,1


In [75]:
# inspect an individual order from prior
order_prods__prior[order_prods__prior.order_id == 1483]

,order_id,product_id,add_to_cart_order,reordered
14111,1483,32818,1,1
14112,1483,27582,2,1
14113,1483,12302,3,1
14114,1483,1831,4,1
14115,1483,19204,5,1
14116,1483,10180,6,1
14117,1483,1545,7,1
14118,1483,18352,8,1
14119,1483,16464,9,1
14120,1483,5531,10,0


### Train

In [107]:
print(order_prods__train.shape)
order_prods__train.head()

(3386, 4)


,order_id,product_id,add_to_cart_order,reordered
239,988,45061,1,0
240,988,28464,2,0
241,988,12626,3,0
242,988,4818,4,1
1202,3529,17261,1,0


In [72]:
# inspect an individual order from train
order_prods__train[order_prods__train.order_id == 988]

,order_id,product_id,add_to_cart_order,reordered
239,988,45061,1,0
240,988,28464,2,0
241,988,12626,3,0
242,988,4818,4,1


## Merge Prior and Train Orders

In [106]:
all_order_prods = pd.concat([order_prods__prior, order_prods__train])
print(all_order_prods.shape)
all_order_prods.head()

(80811, 4)


,order_id,product_id,add_to_cart_order,reordered
333,40,10070,1,1
334,40,42450,2,1
335,40,33198,3,1
336,40,34866,4,1
14111,1483,32818,1,1


# User Database and Features

## Initialize User Dataframe

In [88]:
# Create a users database with some historical information for each user

user_hist = pd.DataFrame()
user_hist['user_id'] = user_sample_list
user_hist.set_index('user_id', inplace=True)
user_hist.head()

""
user_id
1
2
3
4
5


## Generate High-level User Features

### Unique Order Count (User)

In [109]:
def count_user_orders(user_id, orders):
    return int(orders[orders.user_id == user_id].shape[0])

user_hist['order_count'] = pd.Series(user_hist.index.values).apply(count_user_orders, orders=orders)
user_hist.head()

,order_count
user_id,
1,15.0
2,13.0
3,6.0
4,5.0
5,4.0


### Order Sizes (User)

In [128]:
def order_sizes(user_id, orders, order_prods, subset='prior'):
    
    user_orders = orders[(orders.user_id == user_id) & (orders.eval_set == subset)]
    order_id_list = user_orders.order_id.values
    
    order_sizes = []
    for order_id in order_id_list:
        
        prods = order_prods[order_prods.order_id == order_id].product_id.values
        order_sizes += [len(prods)]
        
    return order_sizes

user_hist['prior_order_sizes'] = pd.Series(user_hist.index.values).apply(
    order_sizes,
    orders=orders,
    order_prods=all_order_prods
)

user_hist.head()

,order_count,prior_order_sizes
user_id,,
1,15.0,"[13, 6, 5, 13, 13, 21, 14, 16, 26, 9, 15, 19, ..."
2,13.0,"[10, 9, 6, 5, 11, 8, 9, 8, 5, 6, 5, 6]"
3,6.0,"[4, 2, 7, 2, 3]"
4,5.0,"[11, 9, 5, 12]"
5,4.0,"[4, 7, 3]"


In [130]:
user_hist.order_count.describe()

count    499.000000
mean      16.294589
std       16.066734
min        4.000000
25%        6.000000
50%       11.000000
75%       19.500000
max      100.000000
Name: order_count, dtype: float64

In [133]:
user_hist.prior_order_sizes.describe()

count              499
unique             499
top       [2, 1, 2, 2]
freq                 1
Name: prior_order_sizes, dtype: object

### Average Products / Order (User)

In [135]:
user_hist['avg_ppo'] = user_hist.prior_order_sizes.apply(np.mean)
user_hist.head()

,order_count,prior_order_sizes,avg_ppo
user_id,,,
1,15.0,"[13, 6, 5, 13, 13, 21, 14, 16, 26, 9, 15, 19, ...",13.928571
2,13.0,"[10, 9, 6, 5, 11, 8, 9, 8, 5, 6, 5, 6]",7.333333
3,6.0,"[4, 2, 7, 2, 3]",3.600000
4,5.0,"[11, 9, 5, 12]",9.250000
5,4.0,"[4, 7, 3]",4.666667


In [ ]:
d

### Departmental Unique Check-ins Dictionary (User)

In [ ]:
# Create a dictionary with department IDs as keys
# each value is the number of unique department check-ins.
# i.e. An order with two products from the same department
# should count as 1 unique check-in.

user_hist['dept_dict'] = dict(zip(depts.department_id.values,0*depts.department_id.values))

def unique_dept_checkins(user_id, orders, order_prods, subset='prior'):
    
    user_orders = orders[(orders.user_id == user_id) & (orders.eval_set == subset)]
    
    gb_order = user_orders.groupby('order_id')
    
    
    
    for group in gb_order:
        
        
    
    
    return order_sizes


user_hist['prior_order_sizes'] = pd.Series(user_hist.index.values).apply(
    order_sizes,
    orders=orders,
    order_prods=all_order_prods
)

### Total Orders to Date (User)

### Unique Orders (User)

In [80]:
# add a column which counts total unique orders
# 

In [1]:
import pandas as pd

aisles_df = pd.read_csv('../data/instacart-market-basket-analysis/aisles.csv')
# aisles_df = pd.read_csv('../data/instacart-market-basket-analysis/aisles.csv')


In [ ]:
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

params = {
    'host': '18.220.115.81',
    'user': 'ubuntu',
    'port': 5432
    
# Connect and create database, disconnect, and reconnect to the right database
connection = connect(**params, dbname='ubuntu')
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
connection.cursor().execute('CREATE DATABASE store;')
connection.close()